In [2]:
from collections import Counter,OrderedDict
import csv
import string
import math

In [2]:
reviews, labels = [], []

with open('imdb.txt', encoding='UTF-8') as fp:
    reader = csv.reader(fp)
    next(reader) #skip header

    for row in reader:
        reviews.append(row[0])
        labels.append(row[1])

NameError: name 'csv' is not defined

In [3]:
def tokenize(docs):
    """
    Compute the tokens for each document.

    :param docs: list of strings. Each item is a document to tokenize.
    :return: list of lists. Each item is a list containing the tokens of the relative doc
    """
    tokens = []

    for doc in docs:
        for puncts in string.punctuation:
            doc = doc.replace(puncts, ' ')

        split_doc = [token.lower() for token in doc.split() if token]
        tokens.append(split_doc)

    return tokens

In [4]:
def get_tf(doc):
    tf = {}

    for token in doc:
        if token not in tf:
            tf[token] = 0
        tf[token] += 1

    return tf

def compute_TF(docs):
    tf_docs = []

    for doc in docs:
        tf_docs.append(get_tf(doc))

    return tf_docs

In [5]:
reviews = tokenize(reviews)
tf_reviews = compute_TF(reviews)

In [6]:
# def get_df(token, docs):
#     c = 0
#     for doc in docs:
#         if token in doc:
#             c +=1
#     return c
#
# def compute_DF(docs):
#     toks = {}
#
#     for doc in docs:
#         for token in doc:
#             if token not in toks:
#                 toks[token] = get_df(token, docs)
#
#     return docs
#
# def compute_IDF(docs, tdf):
#     N = len(docs)
#
#     for key in tdf.keys():
#         tdf[key] = math.log(N/tdf[key])
#
#     return tdf
#
# def sort_IDF(tdf):
#     dict1 = OrderedDict(sorted(tdf.items()))
#     print(dict1)

In [7]:
def compute_idf(docs):
    DF = {}
    N = len(docs)

    # compute the document-frequency (DF), i.e. the number of documents in which each token appears at least once
    for doc in docs:
        for token, token_tf in doc.items():
            DF[token] = DF.get(token, 0) +1

    # compute actual IDF
    return {token : math.log(N/df) for token, df in DF.items()}

df_tokens = compute_idf(tf_reviews)

In [8]:
sorted_view = sorted(df_tokens.items(), key = lambda item: item[1])
# key indicates what to take in consideration for the sorting, the key or the
# value of the pair returned by the dictionary.items()

In [9]:
sorted_view[:10]

[]

In [10]:
def norm(d):
    return sum([value**2 for t, value in d.items()])**.5

def dot_product(v1, v2):
    # only the words that appear in at least one of the two vectors/documents are involved
    dict_d = set(list(v1.keys()) + list(v2.keys()))
    return sum([(v1.get(d, 0.0) * v2.get(d, 0.0)) for d in dict_d])

def cosine_similarity(v1, v2):
    return dot_product(v1, v2) / (norm(v1) * norm(v2))

In [13]:
print('Document (0) and document (1) have cosine similarity:', cosine_similarity(tf_reviews[0], tf_reviews[1]))

IndexError: list index out of range

In [12]:

"""
We can now compute the similarities and then assign the class label accordingly.
In order to optimize the evaluation we can exploit the fact that the cosine similarity is a
commutative operator, i.e.  cos(V1,V2)=cos(V2,V1) . In this sense, we can encode similarities
between each pair of reviews in a symmetric matrix and then use it to compute the average with
respect to the positive and negative sets.
"""
import numpy as np

pos_i = [i for i, label in enumerate(labels) if label == '1']
neg_i = [i for i, label in enumerate(labels) if label == '0']
print(len(pos_i), len(neg_i))

similarities = []
y_true = labels
y_pred = []
r_len = len(df_tokens)
try:
    for i, r1 in enumerate(df_tokens):
        store_sim = []
        curr_sim = []
        for j, r2 in enumerate(df_tokens):
            if j == i:
                curr_sim.append(-1) # this value will never be used
            elif j < i:
                curr_sim.append(similarities[j][i-j-1]) # reuse the similarities already evaluated
            else:
                s = cosine_similarity(df_tokens[j], df_tokens[i])
                store_sim.append(s)
                curr_sim.append(s)
        similarities.append(store_sim) # store only the similarities computed in this iteration

        if i in pos_i:
            p_mask = pos_i.copy()
            p_mask.pop(i)
            n_mask = neg_i
        else:
            p_mask = pos_i
            n_mask = neg_i.copy()
            n_mask.pop(i)

        p_mean = np.array(curr_sim)[p_mask].mean()
        n_mean = np.array(curr_sim)[n_mask].mean()

        if p_mean > n_mean:
            y_pred.append('1')
        else:
            y_pred.append('0')

        print(f'{100*i/(r_len):.2f}%', end='\r')
except KeyboardInterrupt:
    print('\nInterrupted')
    pred_c = len(y_pred)
    correct = sum([1 for t, p in zip(y_pred, y_true[:pred_c]) if t == p])
    print(f'Computed {i} reviews up to now. Accuracy: {correct/pred_c * 100:.2f}%')

0 0
